In [1]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, MobileBertTokenizer, MobileBertForSequenceClassification
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

2023-02-19 19:50:33.110194: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
DataSet_name = 'Data_V1.2.csv'
# Model_name = 'roberta'
Model_name = 'mobilebert'
Learning_rate = 5e-5
batch_size = 64
epochs = 1000
is_train = True
word_col = 'Word'
input_col=['Date', 'Month', 'Day', 'WeekNum', 'Contest number']
output_col=['Number of  reported results', 'Number in hard mode', '1 try', '2 tries', '3 tries', '4 tries', '5 tries', '6 tries', '7 or more tries (X)']
model_ckpt_path = 'model_ckpt'

In [3]:
# Load the dataset from a CSV file
df = pd.read_csv(DataSet_name)

# Percentage needs to be divided by 100
Need_div_cols=['1 try', '2 tries', '3 tries', '4 tries', '5 tries', '6 tries', '7 or more tries (X)']
for col in Need_div_cols:
    df[col] = df[col]/100

# Print the first 10 rows of the data
print(df.head(10))

         Date  Month  Day  WeekNum  Contest number   Word  \
0  2022/12/31     12   31        7             560  manly   
1  2022/12/30     12   30        6             559  molar   
2  2022/12/29     12   29        5             558  havoc   
3  2022/12/28     12   28        4             557  impel   
4  2022/12/27     12   27        3             556  condo   
5  2022/12/26     12   26        2             555  judge   
6  2022/12/25     12   25        1             554  extra   
7  2022/12/24     12   24        7             553  poise   
8  2022/12/23     12   23        6             552  aorta   
9  2022/12/22     12   22        5             551  excel   

   Number of  reported results  Number in hard mode  1 try  2 tries  3 tries  \
0                        20380                 1899   0.00     0.02     0.17   
1                        21204                 1973   0.00     0.04     0.21   
2                        20001                 1919   0.00     0.02     0.16   
3       

In [4]:


# Set up the tokenizer and model
if Model_name == 'roberta':
    # # Define the loss function and metric
    # loss_fn = torch.nn.CrossEntropyLoss()
    # metric_fn = accuracy_score

    # Define a new linear layer to map the output to the desired shape
    class CustomOutputLayer(torch.nn.Module):
        def __init__(self, in_features):
            super().__init__()
            self.linear1 = torch.nn.Linear(in_features, 2)
            self.linear2 = torch.nn.Linear(in_features, 7)
            self.softmax = torch.nn.Softmax(dim=1)
        
        def forward(self, x):
            # out1, out2 = x
            out1 = self.linear1(x)
            out2 = self.linear2(x)
            probs = self.softmax(out2)
            return torch.cat([out1, probs], dim=1)

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaForSequenceClassification.from_pretrained('roberta-base')

    # Replace the last layer of the model with the custom output layer
    print(model.classifier)
    # print(model.classifier.in_features)
    model.classifier = CustomOutputLayer(2)

elif Model_name == 'mobilebert':
    # Define a new linear layer to map the output to the desired shape
    class CustomOutputLayer(torch.nn.Module):
        def __init__(self, in_features):
            super().__init__()
            self.linear1 = torch.nn.Linear(in_features, 2)
            self.linear2 = torch.nn.Linear(in_features, 7)
            self.softmax = torch.nn.Softmax(dim=1)

        def forward(self, x):
            out1 = self.linear1(x)
            out2 = self.linear2(x)
            probs = self.softmax(out2)
            return torch.cat([out1, probs], dim=1)

    tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
    model = MobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased')

    # Replace the last layer of the MobileBERT model with the custom output layer
    print(model.classifier)
    print(model.classifier.in_features)
    model.classifier = CustomOutputLayer(model.classifier.in_features)

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

Linear(in_features=512, out_features=2, bias=True)
512


KeyboardInterrupt: 

In [ ]:
# Prepare dataset and dataloader
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, input_col, output_col):
        self.df = df
        self.tokenizer = tokenizer
        self.input_col = input_col
        self.output_col = output_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        inputs = [row[col] for col in self.input_col]
        inputs = ' '.join(str(x) for x in inputs)
        inputs = self.tokenizer.encode_plus(inputs, padding='max_length', max_length=20, truncation=True, return_tensors='pt')
        labels = [row[col] for col in self.output_col]
        labels = torch.tensor(labels, dtype=torch.float32)
        return inputs['input_ids'][0], inputs['attention_mask'][0], labels

# from sklearn.preprocessing import StandardScaler

# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, data, tokenizer, input_cols, output_cols, max_length):
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.inputs = data[input_cols].to_numpy().astype('float32')
#         self.outputs = data[output_cols].to_numpy().astype('float32')
#         self.n_inputs = len(input_cols)
#         self.n_outputs = len(output_cols)
        
#         # normalize inputs using StandardScaler
#         self.scaler = StandardScaler()
#         self.inputs = self.scaler.fit_transform(self.inputs)
        
#     def __len__(self):
#         return len(self.inputs)
    
#     def __getitem__(self, idx):
#         input_ids, attention_mask = self._encode_inputs(self.inputs[idx])
#         labels = self.outputs[idx]
        
#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask,
#             'labels': labels
#         }
    
#     def _encode_inputs(self, row):
#         tokens = [self.tokenizer.cls_token] + [str(elem) for elem in row] + [self.tokenizer.sep_token]
#         input_ids = self.tokenizer.encode(tokens, add_special_tokens=False, max_length=self.max_length)
#         padding_length = self.max_length - len(input_ids)
#         attention_mask = [1] * len(input_ids) + [0] * padding_length
#         input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length
        
#         return input_ids, attention_mask


dataset = CustomDataset(df, tokenizer, input_col, output_col)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Print the first batch of data
for batch in dataloader:
    # print(batch['input_ids'])
    # print(batch['attention_mask'])
    # print(batch['output'])
    print(batch[0])
    print(batch[1])
    print(batch[2])
    break

tensor([[  101, 16798,  2475,  ...,     0,     0,     0],
        [  101, 16798,  2475,  ...,     0,     0,     0],
        [  101, 16798,  2475,  ...,     0,     0,     0],
        ...,
        [  101, 16798,  2475,  ...,     0,     0,     0],
        [  101, 16798,  2475,  ...,     0,     0,     0],
        [  101, 16798,  2475,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[2.0380e+04, 1.8990e+03, 0.0000e+00, 2.0000e-02, 1.7000e-01, 3.7000e-01,
         2.9000e-01, 1.2000e-01, 2.0000e-02],
        [2.1204e+04, 1.9730e+03, 0.0000e+00, 4.0000e-02, 2.1000e-01, 3.8000e-01,
         2.6000e-01, 9.0000e-02, 1.0000e-02],
        [2.0001e+04, 1.9190e+03, 0.0000e+00, 2.0000e-02, 1.6000e-01, 3.8000e-01,
         3.0000e-01, 1.2000e-01, 2.0000e-02],
        [2.0160e+04, 1.9370e+03, 0.0000

In [ ]:
# Set up the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=Learning_rate)
loss_fn = torch.nn.MSELoss()

In [ ]:
# Train the model

# Record the loss and accuracy for each epoch
train_loss = []
# train_acc = []

if is_train:
    model.train()
    # progress_bar = tqdm(range(epochs), desc='Epoch')
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader):
            # input_ids = batch['input_ids'].to(device)
            # attention_mask = batch['attention_mask'].to(device)
            # labels = batch['output'].to(device)
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = loss_fn(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)

        # Record the loss and accuracy for each epoch
        train_loss.append(avg_loss)

        # Use exponential representation and retain 4 decimal places
        avg_loss_str = '{:.4e}'.format(avg_loss)
        
        print(f'Epoch {epoch+1}, Loss: {avg_loss_str}')

        if (epoch+1) % 10 == 0:
            model.save_pretrained(model_ckpt_path+'/epoch'+str(epoch+1)+' loss'+avg_loss_str)


        # train_acc.append(avg_acc)

    # Save the fine-tuned model
    model.save_pretrained(model_ckpt_path+'/final epoch:'+str(epoch+1)+' loss:'+avg_loss_str)

    # Plot the loss and accuracy for each epoch
    import matplotlib.pyplot as plt
    plt.plot(train_loss, label='Training loss')
    plt.title('Training loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    # Save the plot
    plt.savefig('loss.png', dpi=1000)

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1, Loss: 7.9778e+12


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 2, Loss: 2.5470e+10


  0%|          | 0/6 [00:00<?, ?it/s]

...

Epoch 298, Loss: 2.3563e+07


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 299, Loss: 2.1798e+07


  0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # Evaluate the model from the saved checkpoint
# model = model.from_pretrained('fine_tuned_model')
# model.eval()

# # Predict the output for a single example
# example = df.iloc[0]
# inputs = [example[col] for col in input_col]
# inputs = ' '.join(str(x) for x in inputs)
# inputs = tokenizer.encode_plus(inputs, padding='max_length', max_length=20, truncation=True, return_tensors='pt')
# input_ids = inputs['input_ids'].to(device)
# attention_mask = inputs['attention_mask'].to(device)
# outputs = model(input_ids, attention_mask=attention_mask)
# print(outputs.logits)